<a href="https://colab.research.google.com/github/Trnkos/test/blob/main/CSD_Counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def analyze_vehicle_probabilities(reference_file, data_file, output_file):
    # Načtení referenčních dat
    ref_df = pd.read_excel(reference_file)
    ref_df = ref_df.dropna()

    # Načtení nových dat
    data_df = pd.read_excel(data_file, sheet_name='radar')

    # Získání unikátních kategorií
    categories = ref_df['skupina_vozidel'].unique()

    # Výstupní seznam
    output_data = []

    # Iterace přes jednotlivé směry
    for direction in ['+', '-']:
        data_dir = data_df[data_df['smer'] == direction]
        unique_lengths = data_dir['delka'].value_counts().reset_index()
        unique_lengths.columns = ['delka', 'pocet']

        # Iterace přes unikátní délky
        for _, row in unique_lengths.iterrows():
            length = row['delka']
            total_count = row['pocet']

            prob_dict = {cat: 0 for cat in categories}
            count_dict = {cat: 0 for cat in categories}
            total_prob = 0

            # Výpočet pravděpodobnosti pro každý druh vozidla
            for _, ref_row in ref_df[ref_df['smer'] == direction].iterrows():
                group = ref_row['skupina_vozidel']
                mean_length = ref_row['střední hodnota (filtrováno)']
                std_dev = ref_row['směrodatná odchylka (filtrováno)']

                if std_dev > 0:
                    prob = norm.pdf(length, mean_length, std_dev)
                else:
                    prob = 0

                prob_dict[group] = prob
                total_prob += prob

            # Normalizace pravděpodobností, aby součet byl 100 %
            if total_prob > 0:
                prob_dict = {k: (v / total_prob) * 100 for k, v in prob_dict.items()}

            # Výpočet počtu vozidel v kategoriích
            count_dict = {k: (v / 100) * total_count for k, v in prob_dict.items()}

            # Přidání dat do výstupního seznamu
            row_data = [length, direction, total_count] + [prob_dict[cat] for cat in categories] + [count_dict[cat] for cat in categories]
            output_data.append(row_data)

    # Převod do DataFrame
    column_names = ['unikátní délka', 'směr', 'celkový počet'] + [f'pravděpodobnost_{cat} (%)' for cat in categories] + [f'počet_{cat}' for cat in categories]
    output_df = pd.DataFrame(output_data, columns=column_names)

    # Seřazení podle délky
    output_df = output_df.sort_values(by=['unikátní délka'])

    # Uložení do Excelu
    output_df.to_excel(output_file, index=False)

# Spuštění analýzy
reference_file = "output2_vstup.xlsx"
data_file = "radar_video_export.xlsx"
output_file = "probability_output.xlsx"
analyze_vehicle_probabilities(reference_file, data_file, output_file)

print("Hotovo! Výsledek uložen do probability_output.xlsx")


Hotovo! Výsledek uložen do probability_output.xlsx
